##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Camadas personalizadas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Recomendamos usar `tf.keras` como uma API de alto nível para criar redes neurais. No entanto, a maioria das APIs do TensorFlow podem ser usadas com a eager execution.


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.config.list_physical_devices('GPU'))

## Camadas: conjuntos comuns de operações úteis

Na maior parte do tempo, ao escrever código para modelos de aprendizado de máquina, você precisa operar a um nível mais alto de abstração que nas operações individuais e na manipulação de variáveis individuais.

Vários modelos de aprendizado de máquina podem ser expressos como a composição e o empilhamento de camadas relativamente simples, e o TensorFlow oferece tanto um conjunto de camadas bastante comuns quanto maneiras fáceis de você escrever suas próprias camadas específicas aos aplicativos, seja do zero ou como a composição de camadas existentes.

O TensorFlow inclui a API [Keras](https://keras.io) completa no pacote tf.keras, e as camadas do Keras são muito úteis ao criar seus próprios modelos.


In [ ]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

Confira a lista completa de camadas pré-existentes na [documentação](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Ela inclui a Dense (uma camada totalmente conectada), Conv2D, LSTM, BatchNormalization, Dropout e várias outras.

In [ ]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

In [ ]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

In [ ]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

## Implementando camadas personalizadas

A melhor maneira de implementar sua própria camada é estender a classe tf.keras.Layer e implementar:

1. `__init__` , onde você pode fazer toda a inicialização independente de entrada
2. `build`, onde você descobre os formatos dos tensores de entrada e pode fazer o resto da inicialização
3. `call`, onde você faz a computação direta

Você não precisa esperar até que `build` seja chamado para criar suas variáveis. Elas também podem ser criadas em `__init__`. No entanto, a vantagem de criá-las no `build` é que isso permite a criação tardia de variáveis com base no formato das entradas para operação da camada. Por outro lado, com a criação de variáveis em `__init__`, os formatos necessários para criar as variáveis precisarão ser especificados explicitamente.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [ ]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [ ]:
print([var.name for var in layer.trainable_variables])

Em geral, o código é mais fácil de ler e manter quando ele usa camadas padrão sempre que possível, já que os outros leitores estarão familiarizados com o comportamento dessas camadas. Se você quiser usar uma camada que não esteja em `tf.keras.layers`, considere criar um [issue no GitHub](http://github.com/tensorflow/tensorflow/issues/new) ou, melhor ainda, enviar um pull request!

## Modelos: composição de camadas

Vários elementos interessantes que se assemelham a camadas nos modelos de aprendizado de máquina são implementados com a composição de camadas existentes. Por exemplo, cada bloco residual em uma resnet é uma composição de convoluções, normalizações de lotes e um atalho. As camadas podem ser aninhadas dentro de outras camadas.

Geralmente, você herda de `keras.Model` quando precisa de métodos de modelo, como: `Model.fit`,`Model.evaluate` e `Model.save` (confira mais detalhes em [Camadas e modelos personalizados do Keras](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

Outro recurso fornecido por `keras.Model` (em vez de `keras.layers.Layer`) é que, além de rastrear as variáveis, o `keras.Model` também rastreia as camadas internas, facilitando a inspeção delas.

Por exemplo, aqui está um bloco da ResNet:

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

No entanto, na maior parte do tempo, os modelos que compõem várias camadas simplesmente chamam uma camada após a outra. Isso pode ser feito com muito pouco código usando `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Próximos passos

Agora, você pode voltar para o notebook anterior e adaptar o exemplo de regressão linear usando camadas e modelos para deixá-lo mais bem estruturado.